<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Forum-Activity-Report" data-toc-modified-id="Forum-Activity-Report-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Forum Activity Report</a></span></li><li><span><a href="#Specify-Report-Duration" data-toc-modified-id="Specify-Report-Duration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Specify Report Duration</a></span></li><li><span><a href="#Get-Forum-Activity-Data" data-toc-modified-id="Get-Forum-Activity-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get Forum Activity Data</a></span></li><li><span><a href="#Print-Markdown-Report" data-toc-modified-id="Print-Markdown-Report-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Print Markdown Report</a></span></li><li><span><a href="#Copy-and-Paste-in-Markdown-Cell" data-toc-modified-id="Copy-and-Paste-in-Markdown-Cell-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Copy and Paste in Markdown Cell</a></span></li><li><span><a href="#Forum-Activity-Report-From-2022-01-31-To-2022-03-31" data-toc-modified-id="Forum-Activity-Report-From-2022-01-31-To-2022-03-31-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Forum Activity Report From 2022-01-31 To 2022-03-31</a></span><ul class="toc-item"><li><span><a href="#Questions-from-the-Community" data-toc-modified-id="Questions-from-the-Community-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Questions from the Community</a></span></li><li><span><a href="#Knowledgebase-Articles" data-toc-modified-id="Knowledgebase-Articles-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Knowledgebase Articles</a></span></li><li><span><a href="#Release-Notes" data-toc-modified-id="Release-Notes-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Release Notes</a></span></li><li><span><a href="#Announcements" data-toc-modified-id="Announcements-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Announcements</a></span></li></ul></li></ul></div>

# Forum Activity Report
The notebook generates a report of posts in specific categories for the duration specified.

# Specify Report Duration
Set the from_date and to_date in the cell below. The MAX_PAST_PAGES specifies maximum paginations in the recent posts (the Discourse API does not take an explicit date range).

In [1]:
### SET FROM AND TO DATE ##########
from_date = '2022-01-31'
to_date = '2022-03-31'
###################################
## INCREASE IT IF DURATION RANGES BEYOND THE RECENT 2 MONTHS
MAX_PAST_PAGES = 6

# Get Forum Activity Data
Obtain the data by making API requests from the forum server.

In [2]:
import requests
import json
import pprint
from requests.exceptions import HTTPError

print('Processing from_date: {}, to_date: {}'.format(from_date, to_date))

URL_PREFIX = 'https://discuss.aerospike.com/t/'
release_posts = []
kb_posts = []
announce_posts = []
question_posts = []

curr_page = 0
curr_page_min_date = None
while (curr_page < MAX_PAST_PAGES and (curr_page_min_date == None or curr_page_min_date >= from_date)):
    try:
        response = requests.get('https://discuss.aerospike.com/latest.json?page='+str(curr_page))

        response.raise_for_status()
        # access JSOn content
        jsonResponse = response.json()
        #parsed = json.loads(jsonResponse)
        #print("Entire JSON response")
        #pp = pprint.PrettyPrinter(indent=1)
        #pp.pprint(jsonResponse)
        #print(json.dumps(parsed))

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    
    topics = jsonResponse['topic_list']['topics']
    
    # relevant fields and formats: 
    #      'bumped_at': '2022-01-29T12:58:04.851Z', 'category_id': 152, 'id': 9090
    #.     'slug': 'aerospike-rest-client-release-1-9-1-december-16-2021',
    #.     'last_poster_username': 'Aerospike_Knowledge'
    #      'title': 'How to add a new node to a cluster approaching utilisation limits (HWM or stop-writes)'

    for topic in topics:
        topic_id = topic['id']
        topic_date = topic['bumped_at'][:10]
        #print('topic date: ', topic_date)
        
        if topic_id != 4526: # reposted old announcement
            if curr_page_min_date == None or topic_date < curr_page_min_date:
                curr_page_min_date = topic_date

            if topic_date > to_date or topic_date < from_date:
                continue
            
        topic_info = { 'id': topic['id'], 'date': topic['bumped_at'][:10], 'title': topic['title'], 
                               'url': URL_PREFIX+topic['slug']+'/'+str(topic['id']) }
        #print("topic info: ", topic_info)

        #.    if category_id == 152 -> process release
        if topic['category_id'] == 152:
            release_posts.append(topic_info)
            continue

        #.    else if category_id == 3 -> process announcements (ignore)
        if topic['category_id'] == 3:
            announce_posts.append(topic_info)
            continue

        #.    else if last_poster_username == Aerospike_Knowledge -> process knowledgebase
        if topic['last_poster_username'] == 'Aerospike_Knowledge':
            kb_posts.append(topic_info)
            continue

        #.    else process questions (may need to check against negative conditions)
        question_posts.append(topic_info)
        
    print('curr_page: {}, curr-page-min-date: {}'.format(curr_page, curr_page_min_date))
    curr_page += 1
print('Done.')

Processing from_date: 2022-01-31, to_date: 2022-03-31
curr_page: 0, curr-page-min-date: 2022-02-16
curr_page: 1, curr-page-min-date: 2022-02-01
curr_page: 2, curr-page-min-date: 2022-01-29
Done.


# Print Markdown Report
Copy and paste the output of the following cell in another markdown cell or a web page.

In [3]:
print('# Forum Activity Report From {} To {}'.format(from_date, to_date))
print('\n## Questions from the Community')
#pp.pprint(question_posts)
for p in sorted(question_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    print('- [{}]({})'.format(p['title'], p['url']))

print('\n## Knowledgebase Articles')
#pp.pprint(kb_posts)
for p in sorted(kb_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    print('- [{}]({})'.format(p['title'], p['url']))

print('\n## Release Notes', end='')
#pp.pprint(release_posts)
from datetime import date
import re
# extract product and version info
prev_date = None
prev_prod = None
for p in sorted(release_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    #print(p['title'])
    prod_ver = re.match('Aerospike\W*(\D+)([0-9.]\S*)', re.sub('version|Release', '', p['title']))
    if not prod_ver:
        continue
    pv_info = prod_ver.groups()
    prod = pv_info[0].rstrip()
    version = pv_info[1]
    if prev_date != p['date']:
        print('\n\n- {}'.format(date.fromisoformat(p['date']).strftime("%B %d")), end='')
    if prev_prod == prod:
        print(', [{}]({})'.format(version, p['url']), end='')
    else:   
        print('\n\t- {} [{}]({})'.format(prod, version, p['url']), end='')
    prev_date = p['date']
    prev_prod = prod
    
print('\n\n## Announcements')
#pp.pprint(announce_posts)
for p in sorted(announce_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    print('- [{}]({})'.format(p['title'], p['url']))


# Forum Activity Report From 2022-01-31 To 2022-03-31

## Questions from the Community
- [Data loss when used asbackup during migrations](https://discuss.aerospike.com/t/data-loss-when-used-asbackup-during-migrations/9274)
- [EAGAIN (Resource temporarily unavailable) error during asrestore](https://discuss.aerospike.com/t/eagain-resource-temporarily-unavailable-error-during-asrestore/9273)
- [Is there a way to reset the evit counter on the stats?](https://discuss.aerospike.com/t/is-there-a-way-to-reset-the-evit-counter-on-the-stats/9272)
- [UDF problem with maps](https://discuss.aerospike.com/t/udf-problem-with-maps/9270)
- [Aerospike tries to connect to dead node](https://discuss.aerospike.com/t/aerospike-tries-to-connect-to-dead-node/8609)
- [Expiration of records is late only on one node (out of six)](https://discuss.aerospike.com/t/expiration-of-records-is-late-only-on-one-node-out-of-six/9266)
- [Bin convergence and Support of multiple counters in XDR](https://discuss.aerospike.co

# Copy and Paste in Markdown Cell 
Copy the abve output in the following markdown cell.

# Forum Activity Report From 2022-01-31 To 2022-03-31

## Questions from the Community
- [Move data from mem backed namespace to disk backed namespace based on ttl](https://discuss.aerospike.com/t/move-data-from-mem-backed-namespace-to-disk-backed-namespace-based-on-ttl/9256)
- [Ranking using score and pagination support in Aerospike](https://discuss.aerospike.com/t/ranking-using-score-and-pagination-support-in-aerospike/9250)
- [Mocking Aerospike in Go](https://discuss.aerospike.com/t/mocking-aerospike-in-go/9245)
- [Is there a way to get only a particular bin of a record in Python client?](https://discuss.aerospike.com/t/is-there-a-way-to-get-only-a-particular-bin-of-a-record-in-python-client/9240)
- [Cluster join issue](https://discuss.aerospike.com/t/cluster-join-issue/9221)
- [How to rewind XDR for a namespace](https://discuss.aerospike.com/t/how-to-rewind-xdr-for-a-namespace/7487)
- [Handling multiple counters in against a Key](https://discuss.aerospike.com/t/handling-multiple-counters-in-against-a-key/9177)
- [Multiple instances on the same server](https://discuss.aerospike.com/t/multiple-instances-on-the-same-server/5079)
- [Will a namespace TRUNCATE command be replicate via XDR?](https://discuss.aerospike.com/t/will-a-namespace-truncate-command-be-replicate-via-xdr/9231)
- [How to search MAPKEY index](https://discuss.aerospike.com/t/how-to-search-mapkey-index/9229)
- [Incoherent error](https://discuss.aerospike.com/t/incoherent-error/9227)
- [Where can I access reference documentation for older versions of Aerospike?](https://discuss.aerospike.com/t/where-can-i-access-reference-documentation-for-older-versions-of-aerospike/9219)
- [TimeoutError: (9, 'Failed to connect', 'src/main/aerospike/as_cluster.c', 395, False)](https://discuss.aerospike.com/t/timeouterror-9-failed-to-connect-src-main-aerospike-as-cluster-c-395-false/9214)
- [Java client read throws "Error Code -2: Unexpected batch key returned: null,,0"](https://discuss.aerospike.com/t/java-client-read-throws-error-code-2-unexpected-batch-key-returned-null-0/9218)
- [Can you upgrade to 5.0+ by skipping the 4.9 jump version?](https://discuss.aerospike.com/t/can-you-upgrade-to-5-0-by-skipping-the-4-9-jump-version/8044)
- [What is the lowest compatible Java Client version for Aerospike Enterprise server 5.7?](https://discuss.aerospike.com/t/what-is-the-lowest-compatible-java-client-version-for-aerospike-enterprise-server-5-7/9217)
- [Seeing increase in read timeouits when receiving XDR updates](https://discuss.aerospike.com/t/seeing-increase-in-read-timeouits-when-receiving-xdr-updates/9209)
- [How does XDR work with map operations](https://discuss.aerospike.com/t/how-does-xdr-work-with-map-operations/9179)
- [Programmatically get migration status](https://discuss.aerospike.com/t/programmatically-get-migration-status/9202)
- [How to insert GEOJSON using Aerospike Rest Client](https://discuss.aerospike.com/t/how-to-insert-geojson-using-aerospike-rest-client/9174)
- [Increase Block Write Size form 128K to 256K](https://discuss.aerospike.com/t/increase-block-write-size-form-128k-to-256k/978)
- [How to troubleshoot/fix "Device overload"?](https://discuss.aerospike.com/t/how-to-troubleshoot-fix-device-overload/486)
- [Aerospike's Disk not getting mount after machine restart](https://discuss.aerospike.com/t/aerospikes-disk-not-getting-mount-after-machine-restart/1293)

## Knowledgebase Articles
- [Node Not Found For Partition error after a full restart of strongly consistent namespace](https://discuss.aerospike.com/t/node-not-found-for-partition-error-after-a-full-restart-of-strongly-consistent-namespace/9225)
- [How To Generate Logs for an Aerospike Support Case](https://discuss.aerospike.com/t/how-to-generate-logs-for-an-aerospike-support-case/9189)
- [How to add a new node to a cluster approaching utilisation limits (HWM or stop-writes)](https://discuss.aerospike.com/t/how-to-add-a-new-node-to-a-cluster-approaching-utilisation-limits-hwm-or-stop-writes/7460)
- [FAQ - XDR Bin Convergence](https://discuss.aerospike.com/t/faq-xdr-bin-convergence/9165)

## Release Notes

- March 03
	- Connect for Presto - Trino [3.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-3-1-0-march-3-2022/9254), [1.8.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-8-0-march-3-2022/9253)
	- C Client [5.2.8](https://discuss.aerospike.com/t/aerospike-c-client-release-5-2-8-march-3-2022/9252)

- February 28
	- Prometheus Exporter [1.5.0](https://discuss.aerospike.com/t/aerospike-prometheus-exporter-release-1-5-0-february-28-2022/9248)

- February 23
	- Java Object Mapper [2.0.4](https://discuss.aerospike.com/t/aerospike-java-object-mapper-version-2-0-4-february-23-2022/9241)

- February 19
	- Connect for Spark [3.3.0_spark3.1](https://discuss.aerospike.com/t/aerospike-connect-for-spark-3-3-0-spark3-1-february-18-2022/9238)

- February 15
	- C# Client [3.9.16](https://discuss.aerospike.com/t/aerospike-c-client-release-3-9-16-february-15-2022/9224)
	- Connect for Presto - Trino [3.0.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-3-0-0-february-15-2022/9223)

- February 09
	- Document API [1.1.3](https://discuss.aerospike.com/t/aerospike-document-api-version-1-1-3-february-9-2022/9203)

- February 08
	- Connect for Presto - Trino [2.0.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-2-0-0-february-4-2022/9199), [1.7.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-7-0-february-4-2022/9198), [1.6.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-6-0-december-22-2021/9108), [1.5.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-5-0-october-24-2021/8922), [1.4.1](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-4-1-september-20-2021/8842), [1.4.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-4-0-september-17-2021/8841), [1.3.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-3-0-august-21-2021/8746), [1.2.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-2-0-august-15-2021/8725), [1.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-1-0-july-1-2021/8592), [1.0.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-0-0-february-17-2021/8235)
	- Connect for Presto - PrestoSQL [1.0.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-prestosql-1-0-0-january-14-2021/8148)

- February 07
	- JDBC Driver [1.5.0](https://discuss.aerospike.com/t/aerospike-jdbc-driver-version-1-5-0-january-6-2022/9197)

- February 04
	- Connect XDR-Proxy [1.2.0](https://discuss.aerospike.com/t/aerospike-connect-xdr-proxy-1-2-0-february-4-2022/9194)
	- Connect for Pulsar - Outbound [2.2.0](https://discuss.aerospike.com/t/aerospike-connect-for-pulsar-outbound-2-2-0-february-4-2022/9193)
	- Connect for Kafka - Outbound [4.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-kafka-outbound-4-1-0-february-4-2022/9192)
	- Connect for JMS - Outbound [3.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-jms-outbound-3-1-0-february-4-2022/9191)
	- Connect for Event Stream Processing (ESP) - Outbound [1.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-event-stream-processing-esp-outbound-1-1-0-february-4-2022/9190)

- January 31
	- Document API [1.1.2](https://discuss.aerospike.com/t/aerospike-document-api-version-1-1-2-january-31-2022/9170)

## Announcements
- [Cross posting to/from other sites (such as Stack Overflow)](https://discuss.aerospike.com/t/cross-posting-to-from-other-sites-such-as-stack-overflow/4526)